### Setup (database and model)

In [ ]:
import peewee as p

# Create (or connect to) the SQLite database file
db = p.SqliteDatabase("example.db")

# Define a simple Customer table
class Customer(p.Model):
    id_customer = p.AutoField(primary_key=True)
    name = p.CharField()
    age = p.IntegerField()
    salary = p.FloatField()
    email = p.CharField(default='default@example.com')  # default if none given
    phone = p.IntegerField(null=True)                   # optional field

    class Meta:
        database = db

db.connect()
db.create_tables([Customer])  # tables always go inside a list

### CREATE

In [ ]:
# Create a row (all fields supplied)
c1 = Customer.create(
    name="Alex",
    age=30,
    salary=55000.0,
    email="alex@example.com",
    phone=5551234
)

# Create another row (use default email & optional phone)
c2 = Customer.create(
    name="Sam",
    age=22,
    salary=42000.0,
    phone=None
)

print(c1.id_customer, c1.name)
print(c2.id_customer, c2.name)

### READ with select(), where(), order_by()

In [ ]:
# Select all rows
for cust in Customer.select():
    print(cust.id_customer, cust.name, cust.age, cust.salary)

# Filtered select (WHERE)
for cust in Customer.select().where(Customer.age < 25):
    print("Under 25:", cust.name)

# Order by salary
for cust in Customer.select().order_by(Customer.salary):
    print("Ascending order:", cust.name, cust.salary)

for cust in Customer.select().order_by(Customer.salary.desc()):
    print("Descending order:", cust.name, cust.salary)

# get customer with the highest salary:
cust = Customer.select().order_by(Customer.salary.desc()).first()
print(cust.name, cust.salary)

### UPDATE

In [ ]:
cust = Customer.get(Customer.id_customer == 1)
# Update a single record then save()
cust.salary = 60000.0
cust.save()


### DELETE

In [ ]:
cust_2 = Customer.get(Customer.id_customer == 2)
cust_2.delete_instance()

### Overriding create() (custom validation example)

In [ ]:
# Redefine class with custom create()
class Customer(p.Model):
    id_customer = p.AutoField(primary_key=True)
    name = p.CharField()
    age = p.IntegerField()
    salary = p.FloatField()
    email = p.CharField(default='default@example.com')
    phone = p.IntegerField(null=True)

    class Meta:
        database = db

    @classmethod
    def create(cls, **kwargs):
        # Custom validation: email must contain '@'
        email = kwargs.get("email", "")
        if "@" not in email:
            print("Email must contain '@'")
        else:
            # run the original version of .create()
            return super().create(**kwargs)

# Example:
Customer.create(name="Taylor", age=33, salary=70000, email="taylor@example.com")